In [8]:
from collections import Counter
from skimage import data,io,filters
from os import listdir,system
import shutil
#from matplotlib import pyplot as plt
#%matplotlib inline
import numpy
from PIL import Image

In [3]:
train_labels_dataset_array = numpy.loadtxt('Experiment2/train_labels_mini_final.txt', dtype=str)

train_labels_dataset_array[1]
#Total images:1354(without augmenatation)
#After augmenatation: 9478
#unique manatees: 1345
#print(tr)

"b'U0400'"

In [4]:
# Read the array from dis
train_dataset_array = numpy.loadtxt('Experiment2/train_mini_final.txt')

# Note that this returned a 2D array!
print(train_dataset_array.shape)

# However, going back to 3D is easy if we know the 
# original shape of the array
train_dataset_array = train_dataset_array.reshape((9478,64,128))

print(train_dataset_array.shape)

(606592, 128)
(9478, 64, 128)


In [5]:
train_labels_one_hot = numpy.zeros((9478,1354))
for i in range(1354):
    #try using the logic to keep i value as same while reading from labels file
    for j in range(7): #7 is no. of augments
        train_labels_one_hot[i*7+j][i] = 1

In [6]:
# Read the array from disk
test_dataset_array = numpy.loadtxt('Experiment2/test_mini_final.txt')

# Note that this returned a 2D array!
print(test_dataset_array.shape)

# However, going back to 3D is easy if we know the 
# original shape of the array
test_dataset_array = test_dataset_array.reshape((81,64,128))

print(test_dataset_array.shape)

(5184, 128)
(81, 64, 128)


In [7]:
# Read the array from disk
test_labels_dataset_array = numpy.loadtxt('Experiment2/test_labels_mini_final.txt', dtype=str)

print(test_labels_dataset_array.shape)

(81,)


In [9]:
test_labels_one_hot = numpy.zeros((81,1354))
for i in range(81):
    temp_arr = numpy.where(train_labels_dataset_array == test_labels_dataset_array[i])
    #print(temp_arr)
    try:
        idx = (temp_arr[0][0]//7)
    except:
        pass
    test_labels_one_hot[i][idx] = 1
#print(test_labels_one_hot)

In [11]:
train_dataset_array = train_dataset_array.reshape(9478,8192)
test_dataset_array = test_dataset_array.reshape(81,8192)

In [12]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 8192])
y_ = tf.placeholder(tf.float32, shape=[None, 1354])

In [14]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [15]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [16]:
def batch_norm(x, n_out, phase_train):
    """
    Batch normalization on convolutional maps.
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope('bn'):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [17]:
x_image = tf.reshape(x, [-1,64,128,1])

In [18]:
phase_train = tf.placeholder(tf.bool, name='phase_train')

In [19]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])


conv1 = conv2d(x_image, W_conv1) + b_conv1
conv1_bn = batch_norm(conv1, 32, phase_train)
h_conv1 = tf.nn.relu(conv1_bn)
h_pool1 = max_pool_2x2(h_conv1)

In [20]:
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])


conv2 = conv2d(h_pool1, W_conv2) + b_conv2
conv2_bn = batch_norm(conv2, 64,  phase_train)
h_conv2 = tf.nn.relu(conv2_bn)

h_pool2 = max_pool_2x2(h_conv2)

In [21]:
W_conv3 = weight_variable([3, 3, 64, 128])
b_conv3 = bias_variable([128])


conv3 = conv2d(h_pool2, W_conv3) + b_conv3
conv3_bn = batch_norm(conv3, 128,  phase_train)
h_conv3 = tf.nn.relu(conv3_bn)

W_conv3_2 = weight_variable([3, 3, 128, 128])
b_conv3_2 = bias_variable([128])


conv3_2 = conv2d(h_conv3, W_conv3_2) + b_conv3_2
conv3_2_bn = batch_norm(conv3_2, 128,  phase_train)
h_conv3_2 = tf.nn.relu(conv3_2_bn)

h_pool3 = max_pool_2x2(h_conv3_2)

In [22]:
W_fc1 = weight_variable([8 * 16 * 128, 4096])
b_fc1 = bias_variable([4096])

h_pool3_flat = tf.reshape(h_pool3, [-1, 8 * 16 * 128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [23]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [24]:
W_fc2 = weight_variable([4096, 1354])
b_fc2 = bias_variable([1354])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
%%time
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv,labels= y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer(), feed_dict={phase_train:True})
top_1_list = []
top_5_list = []
top_10_list = []
top_20_list = []
val = 1
for i in range(15800):
  batch_image = train_dataset_array[(val-1)*203:val*203,:]
  batch_label = train_labels_one_hot[(val-1)*203:val*203,:]  
   #batch = mnist.train.next_batch(50) 
  val=val+1;  
  if val > 79:
    val = 1
  if i%1 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_image, y_: batch_label, keep_prob: 1.0,  phase_train: False})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  if i%79==0:
    #test_accuracy = accuracy.eval(feed_dict={
        #x:test_dataset_array, y_: test_labels_one_hot, keep_prob: 1.0,  phase_train: False})
    #print("step %d, test accuracy %g"%(i, test_accuracy))  
    labels = tf.argmax(test_labels_one_hot, 1)
    
    top_1 = tf.nn.in_top_k(y_conv, labels, 1)
    predictions_1 = sess.run(top_1, feed_dict={ x:test_dataset_array, y_: test_labels_one_hot, keep_prob: 1.0,  phase_train: False})
    top_1_val = numpy.sum(predictions_1)
    print("step %d, top 1 %g"%(i, top_1_val))
    top_1_list.append(top_1_val)
    
    top_5 = tf.nn.in_top_k(y_conv, labels, 5)
    predictions_5 = sess.run(top_5, feed_dict={ x:test_dataset_array, y_: test_labels_one_hot, keep_prob: 1.0,  phase_train: False})
    top_5_val = numpy.sum(predictions_5)
    print("step %d, top 5 %g"%(i, top_5_val))
    top_5_list.append(top_5_val)
    
    top_10 = tf.nn.in_top_k(y_conv, labels, 10)
    predictions_10 = sess.run(top_10, feed_dict={ x:test_dataset_array, y_: test_labels_one_hot, keep_prob: 1.0,  phase_train: False})
    top_10_val = numpy.sum(predictions_10)
    print("step %d, top 10 %g"%(i, top_10_val))
    top_10_list.append(top_10_val)
    
    top_20 = tf.nn.in_top_k(y_conv, labels, 20)
    predictions_20 = sess.run(top_20, feed_dict={ x:test_dataset_array, y_: test_labels_one_hot, keep_prob: 1.0,  phase_train: False})
    top_20_val = numpy.sum(predictions_20)
    print("step %d, top 20 %g"%(i, top_20_val))
    top_20_list.append(top_20_val)

    saver.save(sess, 'model-4_2.ckpt')
  train_step.run(feed_dict={x: batch_image, y_:batch_label, keep_prob: 0.5, phase_train: True})


step 0, training accuracy 0
step 0, top 1 1
step 0, top 5 1
step 0, top 10 2
step 0, top 20 2
step 1, training accuracy 0.0295567
step 2, training accuracy 0
step 3, training accuracy 0
step 4, training accuracy 0
step 5, training accuracy 0
step 6, training accuracy 0
step 7, training accuracy 0
step 8, training accuracy 0
step 9, training accuracy 0
step 10, training accuracy 0
step 11, training accuracy 0
step 12, training accuracy 0
step 13, training accuracy 0
step 14, training accuracy 0
step 15, training accuracy 0
step 16, training accuracy 0
step 17, training accuracy 0
step 18, training accuracy 0
step 19, training accuracy 0
step 20, training accuracy 0
step 21, training accuracy 0
step 22, training accuracy 0
step 23, training accuracy 0
step 24, training accuracy 0
step 25, training accuracy 0
step 26, training accuracy 0
step 27, training accuracy 0
step 28, training accuracy 0
step 29, training accuracy 0
step 30, training accuracy 0
step 31, training accuracy 0
step 32,

In [35]:
print(top_20_list)

[0, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 3, 4, 4, 4, 5, 4, 8, 6, 7, 3, 2, 5, 7, 4, 6, 4, 7, 5, 10, 11, 12, 6, 9, 11, 8, 10, 10, 10, 11, 9, 11, 9, 12, 9, 11, 9, 12, 11, 13, 12, 14, 14, 12, 14, 15, 16, 15, 14, 14, 15, 17, 18, 13, 19, 19, 22, 21, 18, 17, 18, 17, 16, 21, 22, 24, 25, 21, 22, 21, 20, 21, 23, 22, 24, 20, 23, 23, 20, 26, 22, 24, 27, 28, 28, 28, 30, 31, 26, 31, 30, 29, 23, 30, 28, 27, 31, 25, 31, 30, 30, 29, 28, 31, 33, 35, 29, 35, 30, 32, 30, 32, 33, 32, 29, 32, 33, 30, 33, 32, 32, 28, 34, 33, 30, 33, 31, 29, 30, 31, 37, 33, 39, 34, 31, 33, 31, 33, 30, 34, 31, 32, 30, 35, 34, 29, 33, 31, 29, 31, 34, 37, 36, 38, 35, 32, 36, 33, 34, 34, 32, 35, 31, 34, 26, 29, 33, 37, 30, 36, 31, 33, 34, 29, 33, 34, 29, 33, 32, 37, 36, 33, 37, 35, 34, 40, 38, 34, 40, 33]
